In [1]:
import cv2
import numpy as np
import os
import time
import poly_yolo as yolo #or "import poly_yolo_lite as yolo" for the lite version

Using TensorFlow backend.


In [2]:
#load pretrained model
#if you want to detect more objects, lower the score and vice versa
trained_model = yolo.YOLO(model_path='models/poly_yolo.h5', iou=0.5, score=0.3)

W0611 06:54:33.277767 15288 deprecation.py:506] From C:\Users\Hurtik\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0611 06:55:41.579324 15288 deprecation.py:323] From C:\Users\Hurtik\Anaconda3\lib\site-packages\tensorflow_core\python\ops\array_ops.py:1475: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


models/poly_yolo.h5 model, anchors, and classes loaded.


In [3]:
#helper function
def translate_color(cls):
    if cls == 0: return (230, 25, 75)
    if cls == 1: return (60, 180, 75)
    if cls == 2: return (255, 225, 25)
    if cls == 3: return (0, 130, 200)
    if cls == 4: return (245, 130, 48)
    if cls == 5: return (145, 30, 180)
    if cls == 7: return (70, 240, 240)
    if cls == 8: return (240, 50, 230)
    if cls == 9: return (210, 245, 60)
    if cls == 10: return (250, 190, 190)
    if cls == 11: return (0, 128, 128)
    if cls == 12: return (230, 190, 255)
    if cls == 13: return (170, 110, 40)
    if cls == 14: return (255, 250, 200)
    if cls == 15: return (128, 0, 128)
    if cls == 16: return (170, 255, 195)
    if cls == 17: return (128, 128, 0)
    if cls == 18: return (255, 215, 180)
    if cls == 19: return (80, 80, 128)

In [5]:
dir_imgs_name = 'D:/data-shared/cityscapes/leftImg8bit_demoVideo/leftImg8bit/demoVideo/stuttgart_01'  #path_where_are_images_to_clasification
out_path       = 'poly_yolo_predict/' #path, where the images will be saved. The path must exist
list_of_imgs = [root+"/"+name for root, dirs, files in os.walk(dir_imgs_name) for name in files]    
list_of_imgs.sort()

#browse all images
total_boxes = 0
imgs        = 0
for im in range (0, len(list_of_imgs)):
    imgs    += 1
    img     = cv2.imread(list_of_imgs[im])
    overlay = img.copy()
    boxes   = []
    scores  = []
    classes = []
    
    #realize prediciction using poly-yolo
    startx = time.time()
    box, scores, classes, polygons = trained_model.detect_image(img)
    print('Prediction speed: ', 1.0/(time.time() - startx), 'fps')
    
    
    #example, hw to reshape reshape y1,x1,y2,x2 into x1,y1,x2,y2
    for k in range (0, len(box)):
        boxes.append((box[k][1], box[k][0], box[k][3], box[k][2]))
        cv2.rectangle(img, (box[k][1],box[k][0]), (box[k][3],box[k][2]), translate_color(classes[k]), 3, 1)
    total_boxes += len(boxes)
    
    #browse all boxes
    for b in range(0, len(boxes)):
        f              = translate_color(classes[b])    
        points_to_draw = []
        offset         = len(polygons[b])//3
        
        #filter bounding polygon vertices
        for dst in range(0, len(polygons[b])//3):
            if polygons[b][dst+offset*2] > 0.3: 
                points_to_draw.append([int(polygons[b][dst]), int(polygons[b][dst+offset])])
        
        points_to_draw = np.asarray(points_to_draw)
        points_to_draw = points_to_draw.astype(np.int32)
        if points_to_draw.shape[0]>0:
            cv2.polylines(img, [points_to_draw],True,f, thickness=2)
            cv2.fillPoly(overlay, [points_to_draw], f)
    img = cv2.addWeighted(overlay, 0.4, img, 1 - 0.4, 0)
    cv2.imwrite(out_path+str(imgs)+'.jpg', img)
    
print('total boxes: ', total_boxes)
print('imgs: ', imgs)

Prediction speed:  6.2500245870126925 fps
Prediction speed:  20.408550144270304 fps
Prediction speed:  17.857372763732663 fps
Prediction speed:  18.51871605810411 fps
Prediction speed:  17.545131307046827 fps
Prediction speed:  18.18179611334839 fps
Prediction speed:  16.95024409168795 fps
Prediction speed:  17.54505791457339 fps
Prediction speed:  18.182111382286514 fps
Prediction speed:  17.85813307900608 fps
Prediction speed:  18.86827354764411 fps
Prediction speed:  18.18163848297925 fps
Prediction speed:  17.5439571011486 fps
Prediction speed:  18.18140204255024 fps
Prediction speed:  17.54461757262669 fps
Prediction speed:  16.130015267410425 fps
Prediction speed:  17.544324023608247 fps
Prediction speed:  18.51945196285781 fps
Prediction speed:  18.18234784116525 fps
Prediction speed:  17.54329667939586 fps
Prediction speed:  16.130139330613122 fps
Prediction speed:  18.86818866821116 fps
Prediction speed:  16.949901597475076 fps
Prediction speed:  17.857372763732663 fps
Predict

Prediction speed:  17.241636726875107 fps
Prediction speed:  18.868103789541873 fps
Prediction speed:  17.24177847934754 fps
Prediction speed:  18.519043119663024 fps
Prediction speed:  18.51912488685785 fps
Prediction speed:  18.51896135319025 fps
Prediction speed:  10.7527232361454 fps
Prediction speed:  12.195544907115297 fps
Prediction speed:  10.869564315997886 fps
Prediction speed:  12.500607996948094 fps
Prediction speed:  12.345483658283884 fps
Prediction speed:  12.820775854427188 fps
Prediction speed:  15.62537579769697 fps
Prediction speed:  13.513709266884469 fps
Prediction speed:  17.856916605643637 fps
Prediction speed:  20.409543227238 fps
Prediction speed:  14.084676552761659 fps
Prediction speed:  14.706123250399708 fps
Prediction speed:  16.665954090491876 fps
Prediction speed:  19.230752301654256 fps
Prediction speed:  18.181244419013932 fps
Prediction speed:  13.698638727039949 fps
Prediction speed:  19.231281350585515 fps
Prediction speed:  19.231545885967126 fps
P

Prediction speed:  18.86827354764411 fps
Prediction speed:  16.129705117752927 fps
Prediction speed:  18.18282077737413 fps
Prediction speed:  17.857220708446867 fps
Prediction speed:  18.182663129237547 fps
Prediction speed:  20.40825223822499 fps
Prediction speed:  20.833912011166248 fps
Prediction speed:  20.832980678488056 fps
Prediction speed:  20.40924529219989 fps
Prediction speed:  20.000400551234794 fps
Prediction speed:  20.001354309230763 fps
Prediction speed:  19.608806025273612 fps
Prediction speed:  19.608897698903217 fps
Prediction speed:  20.833601557688105 fps
Prediction speed:  20.40845084128884 fps
Prediction speed:  17.85858929924807 fps
Prediction speed:  20.000305180437934 fps
Prediction speed:  20.407656452225023 fps
Prediction speed:  19.608072667093644 fps
Prediction speed:  19.231634066044915 fps
Prediction speed:  20.000400551234794 fps
Prediction speed:  19.231104997707472 fps
Prediction speed:  19.607064389158463 fps
Prediction speed:  17.24177847934754 fps

Prediction speed:  20.00106817228093 fps
Prediction speed:  18.867764282501124 fps
Prediction speed:  18.51863429451943 fps
Prediction speed:  20.407755747475978 fps
Prediction speed:  17.857144681775026 fps
Prediction speed:  17.85767688207294 fps
Prediction speed:  19.229958782832753 fps
Prediction speed:  17.54535148815126 fps
Prediction speed:  18.182111382286514 fps
Prediction speed:  17.54542488308081 fps
Prediction speed:  17.544470796889573 fps
Prediction speed:  18.182111382286514 fps
Prediction speed:  19.23066412966232 fps
Prediction speed:  16.66734486266531 fps
Prediction speed:  16.949148158924125 fps
Prediction speed:  18.18163848297925 fps
Prediction speed:  17.856384450660094 fps
Prediction speed:  18.868528190525033 fps
Prediction speed:  18.867934034494237 fps
Prediction speed:  18.518061969642115 fps
Prediction speed:  20.000877418862597 fps
Prediction speed:  19.608439339326704 fps
Prediction speed:  20.40884805901301 fps
Prediction speed:  19.231457706697967 fps
P

Prediction speed:  16.949285142769394 fps
Prediction speed:  17.857220708446867 fps
Prediction speed:  18.18179611334839 fps
Prediction speed:  17.857981010771915 fps
Prediction speed:  18.181323230440455 fps
Prediction speed:  17.544470796889573 fps
Prediction speed:  16.66760979796856 fps
Prediction speed:  18.5193701927747 fps
Prediction speed:  17.85874137784212 fps
Prediction speed:  17.54388371849838 fps
Prediction speed:  20.000686669082725 fps
Prediction speed:  18.51887958743951 fps
Prediction speed:  19.608164333866277 fps
Prediction speed:  19.231986794442662 fps
Prediction speed:  18.18179611334839 fps
Prediction speed:  18.5187978224108 fps
Prediction speed:  17.54359019403628 fps
Prediction speed:  19.6099978493216 fps
Prediction speed:  18.5187978224108 fps
Prediction speed:  19.60825600149599 fps
Prediction speed:  18.182663129237547 fps
Prediction speed:  17.544250637888485 fps
Prediction speed:  18.18226902085564 fps
Prediction speed:  18.181953746450787 fps
Predictio

Prediction speed:  16.129270928269555 fps
Prediction speed:  18.51871605810411 fps
Prediction speed:  19.608531009527727 fps
Prediction speed:  20.83349807525146 fps
Prediction speed:  19.23013511468931 fps
Prediction speed:  20.000400551234794 fps
Prediction speed:  19.6098144823459 fps
Prediction speed:  19.231369528237437 fps
Prediction speed:  18.518061969642115 fps
Prediction speed:  18.86784915811587 fps
Prediction speed:  19.99935152249168 fps
Prediction speed:  19.609906165405096 fps
Prediction speed:  18.86759453356245 fps
Prediction speed:  19.231281350585515 fps
Prediction speed:  19.231016822481326 fps
Prediction speed:  18.868782840279277 fps
Prediction speed:  18.868443308801037 fps
Prediction speed:  19.608806025273612 fps
Prediction speed:  19.60871435250117 fps
Prediction speed:  18.86818866821116 fps
Prediction speed:  18.518143728173 fps
Prediction speed:  18.86827354764411 fps
Prediction speed:  20.001163550354548 fps
Prediction speed:  20.000495922941205 fps
Predic